In [75]:
from sklearn.externals import joblib
import random
import numpy as np

In [2]:
events_num, weibos_num, event_weibos_num_list, global_sampling_factor, event_sampling_factor_list = joblib.load(
    'file/pkl/event_sampling_factor.pkl')

In [3]:
events_num

5979

In [4]:
weibos_num

158299

In [6]:
event_weibos_num_list[0:20]

[19, 42, 1, 14, 6, 162, 23, 39, 36, 6, 50, 31, 1, 10, 40, 1, 34, 1, 20, 22]

In [7]:
global_sampling_factor

0.21864320052558767

In [9]:
event_sampling_factor_list[0:20]

[0.0001200260266963152,
 0.00026532069059185463,
 6.317159299806063e-06,
 8.844023019728489e-05,
 3.790295579883638e-05,
 0.0010233798065685821,
 0.00014529466389553945,
 0.00024636921269243644,
 0.00022741773479301828,
 3.790295579883638e-05,
 0.00031585796499030317,
 0.00019583193829398796,
 6.317159299806063e-06,
 6.317159299806063e-05,
 0.0002526863719922425,
 6.317159299806063e-06,
 0.00021478341619340615,
 6.317159299806063e-06,
 0.00012634318599612126,
 0.0001389775045957334]

In [27]:
sampling_num_of_event = [weibos_num * global_sampling_factor * factor for factor in event_sampling_factor_list]

In [28]:
sampling_num_of_event[0:20]

[4.154220809986165,
 9.183014422074681,
 0.21864320052558764,
 3.061004807358227,
 1.311859203153526,
 35.4201984851452,
 5.028793612088516,
 8.527084820497917,
 7.871155218921156,
 1.311859203153526,
 10.932160026279384,
 6.777939216293217,
 0.21864320052558764,
 2.1864320052558766,
 8.745728021023506,
 0.21864320052558764,
 7.43386881786998,
 0.21864320052558764,
 4.372864010511753,
 4.810150411562929]

In [29]:
sampling_num_of_event = [int(x + 0.5) for x in sampling_num_of_event]

In [31]:
print(sum(sampling_num_of_event))
sampling_num_of_event[0:20]

34442


[4, 9, 0, 3, 1, 35, 5, 9, 8, 1, 11, 7, 0, 2, 9, 0, 7, 0, 4, 5]

In [32]:
sampling_num_of_event = [1 if x == 0 else x for x in sampling_num_of_event]

In [33]:
print(sum(sampling_num_of_event))
sampling_num_of_event[0:20]

35329


[4, 9, 1, 3, 1, 35, 5, 9, 8, 1, 11, 7, 1, 2, 9, 1, 7, 1, 4, 5]

## userCertify

In [92]:
events_num, weibos_num, event_weibos_num_list, global_sampling_factor, event_sampling_factor_list = joblib.load(
    'file/pkl/event_sampling_factor.pkl')

# 计算在每个事件中抽取的微博数
sampling_num_of_event = [weibos_num * global_sampling_factor * factor for factor in event_sampling_factor_list]
sampling_num_of_event = [int(x + 0.5) for x in sampling_num_of_event]
# sampling_num_of_event = [int(x) for x in sampling_num_of_event]
sampling_num_of_event = [1 if x == 0 else x for x in sampling_num_of_event]

In [93]:
certify_num_of_event = []
for x in sampling_num_of_event:
    certify_num = [22 / 27 * x, 4 / 27 * x, 1 / 27 * x]
    certify_num = [int(x + 0.5) for x in certify_num]
    certify_num = [1 if x == 0 else x for x in certify_num]
    certify_num_of_event.append(certify_num)

In [94]:
certify_num_of_event = []
for x in sampling_num_of_event:
    certify_num = [22 / 27 * x, 4 / 27 * x, 1 / 27 * x]
    certify_num = [int(x + 0.5) for x in certify_num]
    
    updated_certify_num = []
    for num in certify_num:
        rand = random.random()
        if rand >= 0.9:
            num = 1 if num == 0 else num
        updated_certify_num.append(num)
#     certify_num = [1 if x == 0 else x for x in certify_num]
    certify_num_of_event.append(updated_certify_num)

In [95]:
certify_num_of_event[0:20]

[[3, 1, 0],
 [7, 1, 0],
 [1, 0, 0],
 [2, 0, 1],
 [1, 0, 0],
 [29, 5, 1],
 [4, 1, 0],
 [7, 1, 0],
 [7, 1, 0],
 [1, 1, 0],
 [9, 2, 0],
 [6, 1, 0],
 [1, 0, 0],
 [2, 0, 0],
 [7, 1, 0],
 [1, 0, 0],
 [6, 1, 0],
 [1, 0, 0],
 [3, 1, 0],
 [4, 1, 0]]

In [96]:
sampling_num_of_event = [sum(x) for x in certify_num_of_event]

In [97]:
sum(sampling_num_of_event)

35440

In [98]:
sampling_num_of_event[0:20]

[4, 8, 1, 3, 1, 35, 5, 8, 8, 2, 11, 7, 1, 2, 8, 1, 7, 1, 4, 5]

In [99]:
event_weibos_num_array = np.array(event_weibos_num_list)
sorted_index_of_event = np.argsort(event_weibos_num_array)

In [100]:
sorted_index_of_event

array([2133, 2122, 5613, ..., 1344,  954, 5672])

In [101]:
event_weibos_num_list[2133]

1

In [102]:
sampling_num_of_event[2133]

1

In [103]:
certify_num_of_event[2133]

[1, 0, 0]

## event_features

In [113]:
event_features_list = joblib.load('file/pkl/event_features.pkl')

In [114]:
len(event_features_list)

5979

In [115]:
event_features_list[0]

{'certify_0': [0, 5, 6, 7, 9, 10, 11, 14, 15, 16, 18],
 'certify_1': [1, 8],
 'certify_2': [2, 3, 4, 12, 13, 17],
 'id': 'CE04BFD3766B3D6BAA454A4F9DE8BCAD1458697553928',
 'pic_num': [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1],
 'pic_num_0': [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
 'pic_num_1': [0, 1],
 'pic_num_2': [1, 1, 1, 1, 1, 1],
 'weibo_num': 19}